<a href="https://colab.research.google.com/github/vincentjunitio00/Real-and-Fake-Face-Detection/blob/master/fake_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import zipfile

In [ ]:
!wget --no-check-certificate 'https://github.com/vincentjunitio00/Real-and-Fake-Face-Detection/blob/master/real_and_fake_face.zip?raw=true' -O /kaggle/working/real_and_fake_face.zip

source = './real_and_fake_face.zip'

--2020-10-30 10:12:38--  https://github.com/vincentjunitio00/Real-and-Fake-Face-Detection/blob/master/real_and_fake_face.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/vincentjunitio00/Real-and-Fake-Face-Detection/raw/master/real_and_fake_face.zip [following]
--2020-10-30 10:12:38--  https://github.com/vincentjunitio00/Real-and-Fake-Face-Detection/raw/master/real_and_fake_face.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/vincentjunitio00/Real-and-Fake-Face-Detection/master/real_and_fake_face.zip [following]
--2020-10-30 10:12:38--  https://media.githubusercontent.com/media/vincentjunitio00/Real-and-Fake-Face-Detection/master/real_and_fake_face.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 

In [ ]:
zip_ref = zipfile.ZipFile(source, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

In [ ]:
fake_dir = os.path.join('./real_and_fake_face/training/training_fake')
real_dir = os.path.join('./real_and_fake_face/training/training_real')

print('total training fake images:', len(os.listdir(fake_dir)))
print('total training real images:', len(os.listdir(real_dir)))

total training fake images: 850
total training real images: 971


In [ ]:
fake_files = os.listdir(fake_dir)
real_files = os.listdir(real_dir)

print(fake_files[:10])
print(real_files[:10])

['mid_409_1101.jpg', 'mid_119_1111.jpg', 'mid_354_1110.jpg', 'mid_91_1111.jpg', 'easy_145_1000.jpg', 'easy_133_1100.jpg', 'mid_40_1111.jpg', 'hard_62_1010.jpg', 'easy_50_0110.jpg', 'easy_147_1110.jpg']
['real_00440.jpg', 'real_00617.jpg', 'real_00537.jpg', 'real_00503.jpg', 'real_00907.jpg', 'real_00401.jpg', 'real_00179.jpg', 'real_00436.jpg', 'real_00371.jpg', 'real_00967.jpg']


In [ ]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "./real_and_fake_face/training/"
training_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True)

train_generator = training_datagen.flow_from_directory(
TRAINING_DIR,
target_size=(150,150),
class_mode='binary'
)

VALIDATION_DIR = "./real_and_fake_face/validation/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
VALIDATION_DIR,
target_size=(150,150),
class_mode='binary'
)

Found 1821 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=50, validation_data = validation_generator, verbose = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()